In [2]:
using CSV;
using DataFrames;
using Statistics;
using Distributions;
using Flux;
using Flux: glorot_normal,glorot_uniform, logitcrossentropy;
using Flux: @epochs;
using LinearAlgebra: transpose;
using Plots;

df_fake = CSV.read("train_fake_bow.csv",DataFrame);
df_true = CSV.read("train_true_bow.csv",DataFrame);

df_test = CSV.read("test_bow.csv",DataFrame);

rename!(df_fake,[Symbol("col$i") for i in 1:ncol(df_fake)]);
rename!(df_true,[Symbol("col$i") for i in 1:ncol(df_true)]);
rename!(df_test,[Symbol("col$i") for i in 1:ncol(df_test)]);

num_fake = nrow(df_fake);
num_true = nrow(df_true);

col_fake = vec(ones(num_fake,1));
col_true = vec(zeros(num_true,1));

insertcols!(df_fake, ncol(df_fake)+1, :class=>col_fake);
insertcols!(df_true, ncol(df_true)+1, :class=>col_true);

df = append!(df_fake,df_true; cols =:orderequal);

using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,0.4);

X_train = Array(train[:,1:ncol(df)-1]);
y_train = vec(train[:,ncol(df)]);

X_test = Array(test[:,1:ncol(df)-1]);
y_test = vec(test[:,ncol(df)]);

X_unseen = Array(df_test[:,1:ncol(df_test)]);

num_features = ncol(df) - 1

X_all = Array(df[:,1:ncol(df)-1])

LoadError: UndefVarError: shuffle not defined

In [1]:
data, labels = transpose(X_all), y_train

#define layers of neural net
d1 = Dense(num_features, 500, sigmoid)
d2 = Dense(500,84,sigmoid)
d3 = Dense(84,1)

m = Chain(d1,d2,d3)

loss(x, y) = Flux.Losses.mse(m(x), y)

ps = Flux.params(m)

#optimiser
opt = Descent(0.1)

#training the model and calculate losses
@epochs 3 Flux.train!(loss, ps, [(data,labels)], opt, cb = ()println(loss(data,labels)))

result = m(transpose(X_unseen))
result = vec(result)
result = [v > 0.5 ? 1 : 0 for v in result]

df_results = DataFrame(Category = result);
df_results.Id = 1:22449

df_results = df_results[!, [:Id, :Category]]

CSV.write("Results.csv", df_results);

print("output ready")

LoadError: UndefVarError: X_train not defined

In [ ]:
using Plots
plot(1:epochs,loss_training)
title!("Loss at training epoch")

In [43]:
result = m(transpose(X_unseen))
result = vec(result)
result = [v > 0.5 ? 1 : 0 for v in result]

df_results = DataFrame(Category = result);
df_results.Id = 1:22449

df_results = df_results[!, [:Id, :Category]]

CSV.write("Results.csv", df_results);

print("output ready")

"Results.csv"